# Kennard-Stone Algorithm
The Kennard-Stone algorithm was first described in 1969 ([R. W. Kennard, L. A. Stone, *Technometrics*, **1969**,
*11*, 137–148.](https://www.tandfonline.com/doi/abs/10.1080/00401706.1969.10490666)) to select a diverse subset based on distance. 

## Applicability
**Limitations**

Although this algorithm can be used to perform data set splitting for machine learning, the resulting test set covers the same feature space as the training set. On one hand, this maximizes the feature space covered by the training set. On the other hand, the test scores do not represent a realistic scenario for prediction. 

**Diversity Set**

Due to the deterministic nature of the Kennard-Stone algorithm it is very useful to create a so-called diversity set. For example to reduce the number of molecules selected for synthesis to fit capacity while maintaining the distribution of the original set.  


## Imports & Settings

In [2]:
### Imports
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
### Add the utils directory to the path
sys.path.append(os.path.abspath("../utils"))

from chem import KennardStone

In [4]:
np.set_printoptions(precision=2)

# Data
The chemical structures of BRD4 inhibitors are encoded in bit vectors. For this data set **Morgan fingerprints** `rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect` was used. 






In [5]:
### Load the data
df = pd.read_pickle("data/morgan_2048_df.pkl")
df

0     1     2     3     4     5     6     7     8     9     \
ID                                                                          
CHEMBL1232461     0     0     0     0     0     0     0     0     0     0   
CHEMBL1233528     0     1     0     0     0     1     0     0     0     1   
CHEMBL1313432     0     0     0     0     0     0     0     0     1     0   
CHEMBL1344420     0     0     0     0     0     0     0     0     0     0   
CHEMBL1361699     0     0     0     0     0     0     0     0     1     0   
...             ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
CHEMBL5440963     0     0     0     0     0     0     0     0     0     0   
CHEMBL848         0     0     0     0     0     0     0     0     0     0   
CHEMBL9           0     1     0     0     0     0     0     0     0     0   
CHEMBL98          0     0     0     0     0     0     0     0     0     0   
CHEMBL98350       0     0     0     0     0     0     0     0     0     0   

               ...  2038  2039  2040  2041  2042  2043  2044  2045  2046  2047  
ID             ...                                                              
CHEMBL1232461  ...     0     0     0     0     0     0     0     0     0     0  
CHEMBL1233528  ...     0     0     0     0     0     0     0     0     0     0  
CHEMBL1313432  ...     0     0     0     0     0     0     0     0     0     0  
CHEMBL1344420  ...     0     0     0     0     0     0     0     0     0     0  
CHEMBL1361699  ...     0     0     0     0     0     0     0     0     0     0  
...            ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  
CHEMBL5440963  ...     0     0     0     0     0     0     0     0     0     0  
CHEMBL848      ...     0     0     0     0     0     0     0     0     0     0  
CHEMBL9        ...     0     0     0     0     0     0     0     0     0     0  
CHEMBL98       ...     0     0     0     0     0     0     0     0     0     0  
CHEMBL98350    ...     0     0     0     0     0     0     0     0     0     0  

[2397 rows x 2048 columns]

# Method


## Distance Matrix
Euclidean distance will be used. Between two vectors **x** and **y** in an n-dimensional space it is defined as:

$$
\text{Euclidean dist}(x, y) = \sqrt{ \sum_{i=1}^{n} (x_i - y_i)^2 }
$$

For small data sets it can be calculated by broadcasting in an additional dimension. 

In [6]:
### Demo data (4 sample, 2 features)
demo_X = np.array([
    [0, 0],
    [1, 0],
    [0, 1],
    [1, 1],
    ])

### Calculating the Euclidean distance matrix with broadcasting
np.sqrt(((demo_X[None,:,:] - demo_X[:,None,:])**2).sum(axis=-1))

array([[0.  , 1.  , 1.  , 1.41],
       [1.  , 0.  , 1.41, 1.  ],
       [1.  , 1.41, 0.  , 1.  ],
       [1.41, 1.  , 1.  , 0.  ]])

For **binary vectors** (such as Morgan fingerprints used here), the squared Euclidean distance simplifies to the **Hamming distance** — i.e., the number of positions where the vectors differ:

$$
 \sum_{i=1}^{n} (x_i - y_i)^2 = \sum_{i=1}^{n} x_i + \sum_{i=1}^{n} y_i - 2 \sum_{i=1}^{n} x_i y_i
$$
Where:

- $ \sum_{i=1}^{n} x_i $ is the number of 1s in **x**
- $ \sum_{i=1}^{n} y_i $ is the number of 1s in **y**
- $ \sum_{i=1}^{n} x_i y_i $ is the number of **shared 1s** (intersection)

Computing the distance matrix using this method is more efficient for high-dimensional distance matrixes with a large number of samples and features. 


In [7]:
### Calculate the intersection matrix
intersection = demo_X @ demo_X.T

### Count the number of 1s for each compound
X_norm = demo_X.sum(axis=1).reshape(-1, 1)

### Calculate the Hamming distance matrix
hamming = X_norm + X_norm.T - 2 * intersection

### Euclidean distance
np.sqrt(hamming)

array([[0.  , 1.  , 1.  , 1.41],
       [1.  , 0.  , 1.41, 1.  ],
       [1.  , 1.41, 0.  , 1.  ],
       [1.41, 1.  , 1.  , 0.  ]])

# Class

In [40]:

class KennardStone:
    def __init__(self):
        self.distance_matrix = None
    
    def fit(self, df):
        """
        Calculate the distance matrix.

        Parameters
        ----------
        df : pd.DataFrame
            The input DataFrame with binary values (0/1).
        
        Returns
        -------
        self : KennardStone
            The fitted instance of the KennardStone class.

        Raises
        ------
        ValueError
            If the input DataFrame is not binary (0/1).
        """
        ### Save the input DataFrame as attribute
        self.df = df

        ### Convert the DataFrame to a numpy array
        X = df.to_numpy(dtype=np.int32)

        ### Check if the input is binary
        if not np.isin(X, [0, 1]).all():
            raise ValueError("Input must be binary (0/1) for Hamming distance")

        ### Calculate the intersection matrix
        intersection = X @ X.T

        ### Count the number of 1s for each compound
        X_norm = X.sum(axis=1).reshape(-1, 1)

        ### Calculate the Hamming distance matrix
        dist_matrix = X_norm + X_norm.T - 2 * intersection

        ### Euclidean distance matrix if needed
        #dist_matrix = np.sqrt(dist_matrix)

        ### Save the distance matrix as attribute
        self.distance_matrix = pd.DataFrame(dist_matrix, index=df.index, columns=df.index)
        
        return self

    def split(self, subset_size=0.1, warm_start=False, warm_subset=None):
        """
        Define subset based on the Kennard-Stone algorithm.

        Parameters
        ----------
        subset_size : float
            The size of the subset as a fraction of the total dataset size. Default is 0.1.

        """
        ### Check if the distance matrix is calculated
        if self.distance_matrix is None:
            raise ValueError("Distance matrix not calculated. Call fit() first.")

        ### Calculate the number of samples in the test set
        n_total_samples = len(self.df)
        n_sub_samples = int(n_total_samples * subset_size)

        ### Initialize the boolean mask for the subset
        subset_mask = pd.Series(False, index=self.df.index)

        ### Define the first sample (.idxmax() only returns the first occurrence)
        subset_mask.loc[self.distance_matrix.max().idxmax()] = True


        while subset_mask.sum() < n_sub_samples:
            ### Calculate the distance to the subset
            subset_mask.loc[self.distance_matrix[subset_mask].T[~subset_mask].min(axis=1).idxmax()] = True

        return subset_mask


        


# Sandbox

In [41]:
model = KennardStone().fit(df)
mask = model.split()

In [43]:
mask.sum()

239